In [1]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [1]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
from sklearn.model_selection import KFold
from modules.data_structures.model_dataset import ModelDataset
from model_binaries.utils.binary_utils import save_entity, load_entity, cross_val_train, compute_f1, get_subset

2025-02-09 10:46:47.014498: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations

## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v10/model_v10_test.yaml'
yaml_paths = [yaml_path0]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()

Load Train into Memory

In [4]:
# File path to load train, test, and val datasets
train_test_val_folder_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v10"

In [5]:
train_dataset = load_entity(train_test_val_folder_path, "scaled_train.pkl")
val_dataset = load_entity(train_test_val_folder_path, "scaled_val.pkl")
test_dataset = load_entity(train_test_val_folder_path, "scaled_test.pkl")

Create and Train Models

In [ ]:
reduced_train_set = get_subset(train_dataset, 500)

In [9]:
# === STEP 4: MODEL MANAGER
yamls = [yaml_path0]
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

In [ ]:
models = model_manager.create_models(yamls)
model_manager.train(models, [(train_dataset, val_dataset)], save_after_training=True)

Assess

Launch Tensorboard

In [ ]:
%tensorboard --logdir=./logs